In [5]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever
import pandas as pd
import os
import re
from pathlib import Path

# conda create --name pjn 
# pip install git+https://github.com/apohllo/haystack.git@b79c5b099b294ad5f6cf37a01e4c504b438b8018
# conda install  -c conda-forge ipywidgets
# restart jupyter notebook

In [6]:
from haystack.document_stores import FAISSDocumentStore
from haystack.retriever import DensePassageRetriever

document_store = FAISSDocumentStore(faiss_index_factory_str="Flat")



In [7]:
article_directory = 'artykuly'
article_reqex = r'( *Art\. )(\d+)(\.((?! *Art)([\s\S]))*)'
pattern = re.compile(article_reqex)


In [8]:


artykuly = []
for filename in os.listdir(article_directory):
    with open(os.path.join(article_directory, filename), 'r',encoding='utf-8') as file:
        match = re.match(pattern,file.read())
        
        artykul = {
            'content': match.group(3),
            'meta': {
                'name': Path(file.name).stem,
                'artykul_id' : match.group(2)
            }
        }
        artykuly.append(artykul)
        
print(artykuly[1])
document_store.write_documents(artykuly)

{'content': '.\n1. Do zadań ministra właściwego do spraw zdrowia w zakresie wykonywania zadań\n  systemu należy w szczególności:\n   1) sprawowanie nadzoru nad funkcjonowaniem systemu,\n   2) uzgadnianie i zatwierdzanie wojewódzkich planów zabezpieczenia medycznych\n     działań ratowniczych oraz wojewódzkich programów w zakresie ratownictwa\n     medycznego,\n   3) przygotowywanie i aktualizacja przynajmniej raz w roku krajowego planu\n     zabezpieczenia medycznych działań ratowniczych,\n   4) organizacja transportu medycznego z zagranicy obywateli polskich\n     nieposiadających uprawnień do leczenia poza granicami Rzeczypospolitej\n     Polskiej, wymagających transportu ze względu na stan zagrożenia życia,\n     oraz finansowanie tego transportu wyłącznie w granicach Rzeczypospolitej\n     Polskiej, o ile umowy międzynarodowe nie stanowią inaczej,\n   5) finansowanie działalności lotniczych służb ratowniczych w zakresie\n     medycznych działań ratowniczych,\n   6) prowadzenie reje

Writing Documents:   0%|          | 0/24325 [00:00<?, ?it/s]

In [9]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="enelpol/czywiesz-question",
    passage_embedding_model="enelpol/czywiesz-context"
)

document_store.update_embeddings(retriever)


INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find enelpol/czywiesz-question locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded enelpol/czywiesz-question
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find enelpol/czywiesz-context locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded enelpol/czywiesz-context
INFO - haystack.document_stores.faiss -  Updating embed

Updating Embedding:   0%|          | 0/23916 [00:00<?, ? docs/s]

/home/x/anaconda3/envs/pjn2/lib/python3.7/site-packages/haystack/modeling/data_handler/dataset.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  cur_tensor = torch.tensor([sample[t_name] for sample in features], dtype=torch.long)


Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/3920 [00:00<?, ? Docs/s]

In [15]:
# utils

class color: #kolorki
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def print_aritcles(retriever,df,number_of_chosen_articles,top_k=1,falseOnly = False):
    number_of_found_correct=0
    for question_id in range(number_of_chosen_articles):

        candidate_documents = retriever.retrieve(
            query=df.iloc[question_id]['question'],
            top_k=top_k
        )
        
        for i in range(top_k):
            found_correct = df.iloc[question_id]['passage_id'] == candidate_documents[i].meta['name']
            if found_correct:
                number_of_found_correct+=1
                
            
            if falseOnly and found_correct:
                continue
            
            print(f"{color.RED}{df.iloc[question_id]['question']}{color.END}")

            print(f"{color.PURPLE} AnswerId:{i} Score:{candidate_documents[i].score} plik:{candidate_documents[i].meta['name']} artykul_id:{candidate_documents[i].meta['artykul_id']} {color.END}")
            if top_k<5:
                print(f"Znaleziony artykul to ten na podstawie którego układano pytanie:{color.PURPLE}{found_correct}{color.END}")


            print(candidate_documents[i].content)

        print(f'{color.BLUE}$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$${color.BLUE}')
    return number_of_found_correct


In [13]:
df = pd.read_csv('questions1.csv',names=['passage_id', 'question', 'passage'])



In [16]:
res =print_aritcles(retriever,df,number_of_chosen_articles = 30,top_k=1)


Kto wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
 AnswerId:0 Score:0.9139651870684113 plik:2000_1319_8 artykul_id:8 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Zakazane jest nadużywanie pozycji dominującej na rynku właściwym przez
  jednego lub kilku przedsiębiorców.
2. Nadużywanie pozycji dominującej polega w szczególności na:
   1) bezpośrednim lub pośrednim narzucaniu nieuczciwych cen, w tym cen
     nadmiernie wygórowanych albo rażąco niskich, odległych terminów płatności
     lub innych warunków zakupu albo sprzedaży towarów,
   2) ograniczeniu produkcji, zbytu lub postępu technicznego ze szkodą dla
     kontrahentów lub konsumentów,
   3) stosowaniu w podobnych umowach z osobami trzecimi uciążliwych lub
     niejednolitych warunków umów, stwarzających tym osobom zróżnicowane
     warunki konkurencji,
   4) uzależnianiu zawarcia umowy od przyjęcia lub spełnienia przez drugą
     stronę innego świadczenia, niemającego rzeczowego an

Kogo zwalnia ze służby szef BOR?
 AnswerId:0 Score:0.9077520911194422 plik:2001_298_6 artykul_id:6 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Szef BOR podlega ministrowi właściwemu do spraw wewnętrznych.
2. Szef BOR jest przełożonym funkcjonariuszy BOR, zwanych dalej
  "funkcjonariuszami" oraz pracowników zatrudnionych w BOR.
3. Szefa BOR powołuje i odwołuje Prezes Rady Ministrów na wniosek
  ministra właściwego do spraw wewnętrznych.
4. Zastępców Szefa BOR, na wniosek Szefa BOR, powołuje i odwołuje
  minister właściwy do spraw wewnętrznych.
5. W razie opróżnienia stanowiska Szefa BOR, minister właściwy do spraw
  wewnętrznych, do czasu powołania nowego szefa, powierza pełnienie
  obowiązków Szefa BOR jednemu z jego zastępców, na okres nie dłuższy
  niż 3 miesiące.
6. W razie czasowej niemożności sprawowania funkcji przez Szefa BOR,
  minister właściwy do spraw wewnętrznych, do czasu ustania przeszkody w
  sprawowaniu tej funkcji przez dotychczasowego s

Czy prawo do wykonywania zawodu lekarza było kiedyś równoważne prawu do wykonywania zawodu stomatologa?
 AnswerId:0 Score:0.9185778622712835 plik:1998_1115_3 artykul_id:3 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
W ustawie z dnia 5 grudnia 1996 r. o zawodzie lekarza (Dz.U. z 1997 r. Nr 28,
poz. 152 i Nr 88, poz. 554 oraz z 1998 r. Nr 106, poz. 668) wprowadza się
następujące zmiany:
1) w art. 7:
   a) w ust. 1 wyraz "lub" zastępuje się wyrazem "albo",
   b) dodaje się ust. 1a w brzmieniu:
      "1a. Prawo wykonywania zawodu, o którym mowa w ust. 1, przyznawane na
         czas określony w celu odbycia szkolenia podyplomowego lub uzyskania
         stopnia naukowego obejmuje wyłącznie okres szkolenia oraz wskazane
         miejsce odbywania szkolenia.",
   c) ust. 3 otrzymuje brzmienie:
     "3. Cudzoziemcowi można przyznać prawo wykonywania zawodu, o którym mowa
       w ust. 1, 1a i 2, jeżeli ukończył studia medyczne w języku polskim
       albo jeżeli wy

Gdzie mieści się siedziba Agencji Budowy i Eksploatacji Autostrad?
 AnswerId:0 Score:0.9136971609841591 plik:1994_627_7 artykul_id:7 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
 1. Agencja przygotowuje i koordynuje budowę i eksploatację autostrad w zakresie
   określonym ustawą.
 2. Do zadań Agencji należy:
    1) prowadzenie prac studialnych dotyczących autostrad, obejmujących 
      również ocenę ich oddziaływania na środowisko przyrodnicze,
    2) współpraca z organami właściwymi w sprawach zagospodarowania 
      przestrzennego, obrony narodowej, geodezji i gospodarki gruntami, 
      ewidencji gruntów i budynków, scalania i wymiany gruntów, melioracji 
      wodnych, ochrony gruntów rolnych i leśnych, ochrony środowiska oraz 
      ochrony zabytków,
    3) nabywanie gruntów pod autostrady na rzecz Skarbu Państwa,
    4) opracowywanie projektów kryteriów oceny ofert w postępowaniu przetargowym,
    5) prowadzenie przetargów, opracowywanie projektów konc

Czy osoba, która pobrała nienależnie świadczenie rodzinne powinna je zwrócić?
 AnswerId:0 Score:0.9269781074298207 plik:2003_2255_30 artykul_id:30 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
.
1. Osoba, która pobrała nienależnie świadczenia rodzinne, jest obowiązana do
  ich zwrotu.
2. Za nienależnie pobrane świadczenia rodzinne uważa się:
   1) świadczenia rodzinne wypłacone mimo zaistnienia okoliczności powodujących
     ustanie lub zawieszenie prawa do świadczeń rodzinnych albo wstrzymanie
     wypłaty świadczeń rodzinnych w całości lub w części, jeżeli osoba
     pobierająca te świadczenia była pouczona o braku prawa do ich pobierania;
   2) świadczenia rodzinne przyznane lub wypłacone na podstawie fałszywych
     zeznań lub dokumentów albo w innych przypadkach świadomego wprowadzenia w
     błąd przez osobę pobierającą te świadczenia.
3. Należności z tytułu nienależnie pobranych świadczeń rodzinnych ulegają
  przedawnieniu z upływem 3 lat, licząc od dnia 

In [21]:
res1 =print_aritcles(retriever,df,number_of_chosen_articles = 30,top_k=3)

Kto wydaje decyzję o uznaniu praktyki za ograniczającą konkurencję?
 AnswerId:0 Score:0.9139651870684113 plik:2000_1319_8 artykul_id:8 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Zakazane jest nadużywanie pozycji dominującej na rynku właściwym przez
  jednego lub kilku przedsiębiorców.
2. Nadużywanie pozycji dominującej polega w szczególności na:
   1) bezpośrednim lub pośrednim narzucaniu nieuczciwych cen, w tym cen
     nadmiernie wygórowanych albo rażąco niskich, odległych terminów płatności
     lub innych warunków zakupu albo sprzedaży towarów,
   2) ograniczeniu produkcji, zbytu lub postępu technicznego ze szkodą dla
     kontrahentów lub konsumentów,
   3) stosowaniu w podobnych umowach z osobami trzecimi uciążliwych lub
     niejednolitych warunków umów, stwarzających tym osobom zróżnicowane
     warunki konkurencji,
   4) uzależnianiu zawarcia umowy od przyjęcia lub spełnienia przez drugą
     stronę innego świadczenia, niemającego rzeczowego an

W jakim momencie powstaje spółka komandytowo-akcyjna?
 AnswerId:0 Score:0.927551512933173 plik:1996_465_102 artykul_id:102 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
Spółką komandytową jest spółka osobowa mająca na celu prowadzenie
przedsiębiorstwa pod własną firmą, w której wobec wierzycieli za zobowiązania
spółki co najmniej jeden wspólnik odpowiada bez ograniczenia (komplementariusz),
a odpowiedzialność co najmniej jednego wspólnika (komandytariusza) jest
ograniczona.

W jakim momencie powstaje spółka komandytowo-akcyjna?
 AnswerId:1 Score:0.92742654216212 plik:1996_465_114 artykul_id:114 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
  Kto przystępuje do spółki w charakterze komandytariusza odpowiada także za
 zobowiązania spółki istniejące w chwili wpisania go do rejestru.
 

W jakim momencie powstaje spółka komandytowo-akcyjna?
 AnswerId:2 Score:0.9273867249395725 plik:1996_465_105 artykul_id:105 
Znaleziony artykul to ten n

Czy skarga bez wskazania określonego naruszenia prawa bądź interesu prawnego jest zasadna?
 AnswerId:0 Score:0.9228267450472735 plik:1997_675_102 artykul_id:102 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
. §1. Każdy organ postępowania jest obowiązany przedstawić
             uchybienie stanowiące przyczynę nieważności orzeczenia sądowi
             właściwemu do stwierdzenia nieważności.
           §2. Nieważność z mocy samego prawa stwierdza na posiedzeniu z
             urzędu, z inicjatywy organu postępowania albo na wniosek strony,
             sąd apelacyjny w składzie trzech sędziów. O nieważności
             orzeczenia sądu apelacyjnego oraz Sądu Najwyższego orzeka Sąd
             Najwyższy. Sąd Najwyższy orzeka o nieważności wydanego przez
             siebie orzeczenia w składzie siedmiu sędziów, chyba że orzeczenie
             zostało wydane jednoosobowo.
           §3. Wniosek strony o stwierdzenie nieważności orzeczenia, jeżeli nie
           

Kto sporządza projekt krajowej strategii ochrony i zrównoważonego użytkowania różnorodności biologicznej?
 AnswerId:0 Score:0.9237853791908952 plik:2004_880_111 artykul_id:111 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
.
1. Minister właściwy do spraw środowiska sporządza projekt krajowej strategii
  ochrony i zrównoważonego użytkowania różnorodności biologicznej wraz z
  projektem programu działań.
2. Krajową strategię ochrony i zrównoważonego użytkowania różnorodności
  biologicznej, wraz z programem działań, zatwierdza, w drodze uchwały, Rada
  Ministrów.

Kto sporządza projekt krajowej strategii ochrony i zrównoważonego użytkowania różnorodności biologicznej?
 AnswerId:1 Score:0.9226137450697908 plik:2001_627_84 artykul_id:84 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. W celu doprowadzenia do przestrzegania standardów jakości środowiska w
  przypadkach wskazanych ustawą lub przepisami szczególnymi, w drodze aktu
  prawa miej

Czym są negocjacje bez ogłoszenia?
 AnswerId:0 Score:0.9354030532567165 plik:2004_177_57 artykul_id:57 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Zamawiający zaprasza do negocjacji wykonawców, którzy spełniają warunki
  udziału w postępowaniu, w liczbie, określonej w ogłoszeniu, zapewniającej
  konkurencję, nie mniejszej niż 5, a jeżeli wartość zamówienia na roboty
  budowlane przekracza wyrażoną w złotych równowartość kwoty 10 000 000 euro, a
  na dostawy lub usługi - 5 000 000 euro, nie mniejszej niż 7.
2. Jeżeli liczba wykonawców, którzy spełniają warunki udziału w postępowaniu,
  jest większa niż określona w ogłoszeniu, zamawiający zaprasza do negocjacji
  wykonawców, którzy otrzymali najwyższą ocenę spełniania tych warunków.
3. Jeżeli liczba wykonawców, którzy spełniają warunki udziału w postępowaniu,
  jest mniejsza niż określona w ogłoszeniu, zamawiający zaprasza do negocjacji
  wszystkich wykonawców spełniających te warunki.
4. Wraz z zaproszeni

Czy środki ochrony roślin mogą być nakładane poprzez niesprawny technicznie sprzęt?
 AnswerId:0 Score:0.894547285438947 plik:1995_446_32 artykul_id:32 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
Posiadacze są obowiązani do prowadzenia ewidencji wykonywanych zabiegów 
środkami ochrony roślin I i II klasy toksyczności, która powinna obejmować 
terminy ich wykonywania oraz nazwy zastosowanych środków ochrony roślin i 
ich dawki; ewidencja powinna być przechowywana przez okres jednego cyklu 
produkcyjnego.

Czy środki ochrony roślin mogą być nakładane poprzez niesprawny technicznie sprzęt?
 AnswerId:1 Score:0.8943403224929042 plik:1995_446_37 artykul_id:37 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
 1. Środki ochrony roślin na terenie otwartym stosuje się:
    1) sprzętem naziemnym, jeżeli:
       a) prędkość wiatru nie przekracza 3m/s,
       b) miejsce stosowania środka ochrony roślin jest oddalone co najmniej
         5 m od dró

Na czyj wniosek osoba małoletnia może być zwolniona przez policję?
 AnswerId:0 Score:0.9153393353001675 plik:2004_1535_14 artykul_id:14 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
Wniosek o
wydanie dokumentu paszportowego osobie ubezwłasnowolnionej składa
przedstawiciel ustawowy tej osoby lub ustanowiony przez sąd opiekun.
 

Na czyj wniosek osoba małoletnia może być zwolniona przez policję?
 AnswerId:1 Score:0.9151420001833422 plik:2003_2255_7 artykul_id:7 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
Zasiłek rodzinny nie przysługuje, jeżeli dziecko:
   1) przebywa w instytucji zapewniającej całodobowe utrzymanie lub w rodzinie
     zastępczej;
   2) pozostaje w związku małżeńskim.
                                        

Na czyj wniosek osoba małoletnia może być zwolniona przez policję?
 AnswerId:2 Score:0.9149083441027731 plik:2004_2776_18 artykul_id:18 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
. 


Czy żołnierzom zawodowym - kobietom przysługuje urlop macierzyński?
 AnswerId:0 Score:0.9194562195786425 plik:2003_1750_65 artykul_id:65 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
.
1. Żołnierzom zawodowym - kobietom będącym w ciąży i po porodzie przysługuje
  urlop macierzyński na zasadach i w wymiarze określonych w przepisach ustawy z
  dnia 26 czerwca 1974 r. - Kodeks pracy (Dz.U. z 1998 r. Nr 21, poz. 94, z
  późn. zm.)[3].
2. Zasad udzielania urlopu macierzyńskiego nie stosuje się do żołnierzy
  zawodowych - mężczyzn.

Czy żołnierzom zawodowym - kobietom przysługuje urlop macierzyński?
 AnswerId:1 Score:0.9184374932285011 plik:2004_535_65 artykul_id:65 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
. 1.         Żołnierzowi zawodowemu
przysługuje urlop macierzyński na zasadach i w wymiarze określonych w
przepisach ustawy z dnia 26 czerwca 1974 r. – Kodeks pracy (Dz. U.
z 1998 r. Nr 21, poz. 94, z późn. zm.[5])).
2.  Żołnierzowi
zaw

Jakiej ochronie podlegają obiekty jądrowe?
 AnswerId:0 Score:0.8972942476099186 plik:2001_18_34 artykul_id:34 
Znaleziony artykul to ten na podstawie którego układano pytanie:True
.
1. Do obiektów jądrowych należą w szczególności:
   1) elektrownie, elektrociepłownie i ciepłownie jądrowe z energetycznymi
     reaktorami jądrowymi,
   2) badawcze, doświadczalne i inne reaktory jądrowe,
   3) obiekty przeznaczone do wytwarzania, przetwarzania, przechowywania i
     składowania materiałów jądrowych i paliwa jądrowego
   - od rozpoczęcia budowy do zakończenia ich likwidacji.
2. Obiekty jądrowe podlegają ochronie fizycznej.

Jakiej ochronie podlegają obiekty jądrowe?
 AnswerId:1 Score:0.8960072784402946 plik:2001_18_35 artykul_id:35 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
1. Obowiązek spełnienia wymagań bezpieczeństwa jądrowego i ochrony
  radiologicznej oraz ochrony fizycznej obiektu jądrowego w okresie ustalania
  lokalizacji, projektowania, budowy, rozruc

Kto wybiera likwidatora spółdzielni?
 AnswerId:0 Score:0.9287383790934993 plik:2003_1939_85 artykul_id:85 
Znaleziony artykul to ten na podstawie którego układano pytanie:False
.
Likwidator niezwłocznie po wyznaczeniu go:
   1) zgłasza do Krajowego Rejestru Sądowego wniosek o wpisanie otwarcia
     likwidacji spółdzielni, jeżeli takiego zgłoszenia nie dokonano, i
     zawiadamia o tym związek rewizyjny, w którym spółdzielnia jest zrzeszona
     oraz Radę Spółdzielczą,
   2) powiadamia właściwy urząd skarbowy,
   3) ogłasza w Monitorze Spółdzielczym zawiadomienie o otwarciu likwidacji
     spółdzielni i wzywa wierzycieli do zgłoszenia wierzytelności w terminie 3
     miesięcy od dnia ogłoszenia,
   4) przystępuje do sporządzenia sprawozdania finansowego na dzień otwarcia
     likwidacji oraz listy zobowiązań spółdzielni,
   5) sporządza plan finansowy likwidacji, w którym wydziela się odpowiednie
     środki na koszty lustracji, które będą przeprowadzane w okresie likwidacji
     spółdz

In [22]:
print(f'Odnalezione w top1:{res}/30 Odnalezione w top3:{res1}/30')

Odnalezione w top1:7/30 Odnalezione w top3:12/30


In [32]:
#@1

relevant_and_retrieved =  res 
retrieved = 30
relevant = 30
precision = relevant_and_retrieved/retrieved
recall = relevant_and_retrieved/relevant

print(f'precision@1: {precision}  recall@1: {recall}')

precision@1: 0.23333333333333334  recall@1: 0.23333333333333334


In [31]:
#@3
relevant_and_retrieved =  res1
retrieved = 90
relevant = 90
precision = relevant_and_retrieved/retrieved
recall = relevant_and_retrieved/relevant

print(f'precision@3 :{precision}  recall@3 :{recall}')


precision@3 :0.13333333333333333  recall@3 :0.13333333333333333
